# Olo Demo Setup & Data Loading

This notebook handles all the initial setup for the Olo AISQL demo including:
- Database and schema creation
- Table structure setup  
- CSV data loading and verification
- Environment preparation for AISQL functions

**Run this notebook first before proceeding to the AISQL demos.**


## Step 1: Create Database and Schema


In [ ]:
-- Create the Olo demo environment
CREATE OR REPLACE DATABASE OLO_AISQL_DEMO;
USE DATABASE OLO_AISQL_DEMO;
CREATE OR REPLACE SCHEMA RESTAURANT_DATA;
USE SCHEMA RESTAURANT_DATA;

-- Ensure we have a warehouse running
USE WAREHOUSE COMPUTE_WH; -- Change this to your preferred warehouse

-- Verify setup
SELECT CURRENT_DATABASE() as database_name, CURRENT_SCHEMA() as schema_name;


## Step 2: Create Table Structures


In [ ]:
-- Create restaurants table
CREATE OR REPLACE TABLE restaurants (
    restaurant_id INT PRIMARY KEY,
    restaurant_name VARCHAR(100),
    cuisine_type VARCHAR(50),
    address VARCHAR(200),
    city VARCHAR(50),
    state VARCHAR(2),
    zip_code VARCHAR(10),
    phone VARCHAR(20),
    email VARCHAR(100),
    owner_name VARCHAR(100),
    date_joined DATE,
    status VARCHAR(20),
    avg_prep_time_minutes INT,
    delivery_fee DECIMAL(5,2),
    minimum_order DECIMAL(6,2)
);

-- Create customers table
CREATE OR REPLACE TABLE customers (
    customer_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    email VARCHAR(100),
    phone VARCHAR(20),
    address VARCHAR(200),
    city VARCHAR(50),
    state VARCHAR(2),
    zip_code VARCHAR(10),
    date_registered DATE,
    total_orders INT,
    total_spent DECIMAL(10,2),
    loyalty_tier VARCHAR(20),
    preferred_cuisine VARCHAR(50),
    last_order_date DATE
);


In [ ]:
-- Create menu_items table
CREATE OR REPLACE TABLE menu_items (
    item_id INT PRIMARY KEY,
    restaurant_id INT,
    item_name VARCHAR(100),
    category VARCHAR(50),
    description TEXT,
    price DECIMAL(6,2),
    cost DECIMAL(6,2),
    prep_time_minutes INT,
    calories INT,
    is_vegetarian BOOLEAN,
    is_vegan BOOLEAN,
    is_gluten_free BOOLEAN,
    popularity_score DECIMAL(3,1),
    date_added DATE
);

-- Create orders table
CREATE OR REPLACE TABLE orders (
    order_id INT PRIMARY KEY,
    customer_id INT,
    restaurant_id INT,
    order_date DATE,
    order_time TIME,
    order_type VARCHAR(20),
    status VARCHAR(20),
    subtotal DECIMAL(8,2),
    tax DECIMAL(6,2),
    delivery_fee DECIMAL(5,2),
    tip DECIMAL(6,2),
    total_amount DECIMAL(8,2),
    estimated_delivery_time TIME,
    actual_delivery_time TIME,
    payment_method VARCHAR(20),
    promo_code VARCHAR(20),
    driver_id INT
);


In [ ]:
-- Create remaining tables
CREATE OR REPLACE TABLE order_items (
    order_item_id INT PRIMARY KEY,
    order_id INT,
    item_id INT,
    quantity INT,
    unit_price DECIMAL(6,2),
    total_price DECIMAL(8,2),
    special_instructions TEXT,
    customizations TEXT
);

CREATE OR REPLACE TABLE delivery_drivers (
    driver_id INT PRIMARY KEY,
    first_name VARCHAR(50),
    last_name VARCHAR(50),
    phone VARCHAR(20),
    email VARCHAR(100),
    vehicle_type VARCHAR(20),
    license_plate VARCHAR(20),
    date_hired DATE,
    status VARCHAR(20),
    current_rating DECIMAL(2,1),
    total_deliveries INT,
    total_earnings DECIMAL(10,2),
    zone_coverage TEXT
);

CREATE OR REPLACE TABLE promotions (
    promo_id INT PRIMARY KEY,
    promo_code VARCHAR(20),
    description TEXT,
    discount_type VARCHAR(20),
    discount_value DECIMAL(6,2),
    min_order_amount DECIMAL(6,2),
    max_discount DECIMAL(6,2),
    start_date DATE,
    end_date DATE,
    usage_limit INT,
    times_used INT,
    status VARCHAR(20),
    applicable_restaurants TEXT
);


## Step 3: Create Stage and Upload Data

**Manual Step Required:** 
1. Go to Snowflake Web UI → Databases → OLO_AISQL_DEMO → RESTAURANT_DATA → Stages
2. Click on OLO_DATA_STAGE after running the command below
3. Upload all 7 CSV files from your `olo_demo_data` folder


In [ ]:
-- Create stage for CSV files
CREATE OR REPLACE STAGE olo_data_stage;

-- List files after upload (run this after uploading CSV files)
LIST @olo_data_stage;


## Step 4: Load Data from CSV Files

Run these commands in order after uploading the CSV files:


In [ ]:
-- Load restaurants data
COPY INTO restaurants FROM @olo_data_stage/restaurants.csv
FILE_FORMAT = (TYPE = 'CSV' SKIP_HEADER = 1);

-- Load customers data  
COPY INTO customers FROM @olo_data_stage/customers.csv
FILE_FORMAT = (TYPE = 'CSV' SKIP_HEADER = 1);

-- Load menu items data
COPY INTO menu_items FROM @olo_data_stage/menu_items.csv
FILE_FORMAT = (TYPE = 'CSV' SKIP_HEADER = 1);


In [ ]:
-- Load orders and related data
COPY INTO orders FROM @olo_data_stage/orders.csv
FILE_FORMAT = (TYPE = 'CSV' SKIP_HEADER = 1);

COPY INTO order_items FROM @olo_data_stage/order_items.csv
FILE_FORMAT = (TYPE = 'CSV' SKIP_HEADER = 1);

-- Load supporting data
COPY INTO delivery_drivers FROM @olo_data_stage/delivery_drivers.csv
FILE_FORMAT = (TYPE = 'CSV' SKIP_HEADER = 1);

COPY INTO promotions FROM @olo_data_stage/promotions.csv
FILE_FORMAT = (TYPE = 'CSV' SKIP_HEADER = 1);


## Step 5: Verify Data Load


In [ ]:
-- Verify all data loaded correctly
SELECT 'Restaurants' as table_name, COUNT(*) as records FROM restaurants
UNION ALL
SELECT 'Customers', COUNT(*) FROM customers
UNION ALL
SELECT 'Menu Items', COUNT(*) FROM menu_items
UNION ALL
SELECT 'Orders', COUNT(*) FROM orders
UNION ALL
SELECT 'Order Items', COUNT(*) FROM order_items
UNION ALL
SELECT 'Drivers', COUNT(*) FROM delivery_drivers
UNION ALL
SELECT 'Promotions', COUNT(*) FROM promotions
ORDER BY table_name;

-- Expected Results:
-- Customers: 20
-- Drivers: 10  
-- Menu Items: 30
-- Order Items: 52
-- Orders: 30
-- Promotions: 10
-- Restaurants: 15


In [ ]:
-- Sample data preview
SELECT 'Restaurant Sample' as data_type, restaurant_name, cuisine_type, city FROM restaurants LIMIT 3
UNION ALL
SELECT 'Menu Sample', item_name, category, CAST(price AS VARCHAR) FROM menu_items LIMIT 3
UNION ALL  
SELECT 'Order Sample', CAST(order_id AS VARCHAR), order_type, CAST(total_amount AS VARCHAR) FROM orders LIMIT 3;


## ✅ Setup Complete!

Your Olo demo environment is now ready. You can proceed to:

1. **02_Olo_AISQL_Functions.ipynb** - Core AISQL function demonstrations
2. **03_Olo_Cortex_Search.ipynb** - Cortex Search Service setup and examples  
3. **04_Olo_Cortex_Analyst.ipynb** - Natural language query capabilities

### Troubleshooting

If you encounter issues:

```sql
-- Check for any load errors
SELECT * FROM TABLE(INFORMATION_SCHEMA.COPY_HISTORY(TABLE_NAME=>'RESTAURANTS', START_TIME=> DATEADD(hours, -1, CURRENT_TIMESTAMP())));

-- Verify Cortex functions are available
SELECT SYSTEM$GET_CORTEX_FUNCTIONS();

-- Test basic Cortex functionality
SELECT SNOWFLAKE.CORTEX.COMPLETE('llama3-8b', 'Hello, this is a test.');
```
